In [240]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [241]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [242]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [243]:
%%capture pwd
!pwd

In [244]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('update-how-is-scotlands-economy-faring-in-the-pandemic', 300, 500)

# Fig 2

In [245]:
df = pd.read_csv("raw/data_2022-Feb-06-shr.csv")
df = (
    df.set_index(["date", "areaName"])[
        [
            "cumVaccinationSecondDoseUptakeByPublishDatePercentage",
            "cumVaccinationThirdInjectionUptakeByPublishDatePercentage",
        ]
    ]
    .stack()
    .reset_index()
)
df.columns = ["date", "region", "dose", "value"]
df["dose"] = (
    df["dose"]
    .replace("cumVaccinationThirdInjectionUptakeByPublishDatePercentage", "3rd")
    .replace("cumVaccinationSecondDoseUptakeByPublishDatePercentage", "2nd")
)

In [246]:
f = "fig2_vac"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,region,dose,value
0,2022-02-05,England,2nd,84.2
1,2022-02-05,England,3rd,64.8
2,2022-02-04,England,2nd,84.2
3,2022-02-04,England,3rd,64.8
4,2022-02-03,England,2nd,84.1


In [247]:
base = (
    alt.Chart(f2)
    .encode(
        x=alt.X(
            "date:T",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.6,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=5,
                orient="bottom",
                labelAngle=0,
            ),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.region+' | '+utcFormat(datum.date,'%B %d, %Y')+' | '+datum.dose+ ' dose: '+datum.value +'%'"
    )
)
area = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        line={"color": colors["eco-turquiose"]},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color=colors["eco-turquiose"], offset=0.7),
                alt.GradientStop(color="white", offset=0),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(
        y=alt.Y(
            "y:Q",
            sort=[],
            stack=False,
            axis=alt.Axis(
                title="",
                #             domain=False,
                gridOpacity=0.4,
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=21,
                format=".0%",
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleFontWeight="normal",
                titleAlign="left",
                tickCount=6,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        row=alt.Row(
            "region:N",
            header=alt.Header(
                title="", orient="top", labelAnchor="middle", labelPadding=-70
            ),
        ),
        opacity=alt.Opacity(
            "dose:N", title="Dose", legend=alt.Legend(orient="top-left", padding=85)
        ),
    )
    .transform_calculate(y="datum.value/100")
)
layer1 = (
    ((area).properties(height=90, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

WARN Channel opacity should not be used with an unsorted discrete field.
WARN Channel opacity should not be used with an unsorted discrete field.
WARN Channel opacity should not be used with an unsorted discrete field.


alt.Chart(...)

# Fig 4

In [248]:
df = (
    pd.read_excel(
        "raw/mGDP+-+October+2021+-+Publication+Tables_FIGURE_4.xlsx",
        sheet_name="Sheet1",
    )
    .drop("Unnamed: 0", axis=1)
    .T.reset_index()
)
df.columns = ["industry", "y", "x", "size"]

In [249]:
f = "fig4_industry"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,industry,y,x,size
0,"Agriculture, forestry and fishing",-0.008956,-0.079572,14.762448
1,Mining and Quarrying Industries,-0.169447,-0.365562,11.671715
2,Manufacturing,-0.006915,-0.227589,103.712348
3,Electricity & Gas Supply,-0.056559,-0.134261,26.878995
4,Water Supply & Waste Management,0.039964,-0.173648,16.036054


In [250]:
base = (
    alt.Chart(f4)
    .encode(
        x=alt.X(
            "x:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="right",
                titleAnchor="end",
                title="GDP change Feb 2020 - Apr 2020",
                titleY=-15,
                titleX=300,
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        ),
        y=alt.Y(
            "y:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="GDP change Feb 2020 - Oct 2021",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        fill=alt.Color(
            "industry:N",
            legend=None,
            scale=alt.Scale(range=category_color + list(colors.values())),
        ),
        color=alt.Color(
            "industry:N",
            legend=None,
            scale=alt.Scale(range=category_color + list(colors.values())),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.industry+' | Feb - Apr 2020: '+round(datum.x*1000)/10+'% | Feb 2020 - Oct 2021: '+round(datum.y*1000)/10+ '%'"
    )
)

points = base.mark_point(fillOpacity=0.7, opacity=0.4).encode(
    size=alt.Size("size:Q", legend=None, scale=alt.Scale(domain=[0, 20]))
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(x=alt.X("x:Q", sort=[]))
)
axis2 = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y=alt.Y("y:Q", sort=[]))
)
text1 = (
    base.mark_text(fontSize=11)
    .encode(text="industry:N")
#     .transform_filter(alt.FieldOneOfPredicate(field="industry", oneOf=["Construction"]))
    .transform_filter("datum.industry!='Electricity & Gas Supply'")
    .transform_filter("datum.industry!='Information & Communication'")
    .transform_filter("datum.industry!='Financial & Insurance Activities'")
    .transform_filter("datum.industry!='Agriculture, forestry and fishing'")
    .transform_filter("datum.industry!='Real Estate Activities'")
)
text2 = (
    base.mark_text(fontSize=11,dy=10)
    .encode(text="industry:N")
    .transform_filter("datum.industry=='Electricity & Gas Supply'")
)
text3 = (
    base.mark_text(fontSize=11,dy=-10)
    .encode(text="industry:N")
    .transform_filter("datum.industry=='Information & Communication'")
)
text4 = (
    base.mark_text(fontSize=11,dx=55)
    .encode(text="industry:N")
    .transform_filter("datum.industry=='Real Estate Activities'")
)
text5 = (
    base.mark_text(fontSize=11,dy=13)
    .encode(text="industry:N")
    .transform_filter("datum.industry=='Agriculture, forestry and fishing'")
)
text6 = (
    base.mark_text(fontSize=11,dy=19)
    .encode(text="industry:N")
    .transform_filter("datum.industry=='Financial & Insurance Activities'")
)
layer1 = (
    ((axis1 + axis2 + points + text1+text2+text3+text4+text5+text6).properties(height=400, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

WARN Dropping color encoding as the plot also has fill.
WARN Dropping color encoding as the plot also has fill.
WARN Dropping color encoding as the plot also has fill.
WARN Dropping color encoding as the plot also has fill.
WARN Dropping color encoding as the plot also has fill.
WARN Dropping color encoding as the plot also has fill.


alt.LayerChart(...)